In [2]:
# Import libraries
import numpy as np
from scipy import sparse
import pandas as pd
import ipynb

beta = 0.91 # used in computing the PageRank

In [3]:
# load the Graph2022.csv and store it in variable "M" as a np.array (the loading may take some time)

# You may use the line below to load the data.
M = pd.read_csv("Graph2022.csv", header = None).values

<div class="alert alert-warning">
<b>M is a square matrix contains only 0 and 1. $M_{ij}=1$ if and only if webpage j links to webpage i. The number of rows (or columns) of the matrix is the number of webpages in the network. You may want to store that number in a variable<b>
</div>

In [4]:
M.shape

(9998, 9998)

In [5]:

print('number of links ' + str(M.sum()))
print('sparse ratio ' + str(1 - M.sum()/(len(M)**2)))

number of links 271976
sparse ratio 0.9972791517695417


In [6]:
# Check if there are any NaN in your matrix after the operation
test = M.sum(axis=0)[None,:]

test[np.where(test==0)]=1

M2 = M/test
np.isnan(M2).any()

False

In [7]:
H = M2
for i in range(len(H)):
    if H[:,i].sum() == 0:
        H[:,i] = 1/9998

np.where(H.sum(axis=0)[None,:]==0)

(array([], dtype=int64), array([], dtype=int64))

In [9]:
# Generate matrix G using beta. 
beta_1 = beta*H
beta_2 = (1 - beta)*np.ones((9998,9998))/9998

G = beta_1 + beta_2

In [10]:
# initialize the PageRank vector to be 1/N for all entries, where N is the total number of webpage in the graph
M_int = np.ones((9998,1))/9998
M_int

array([[0.00010002],
       [0.00010002],
       [0.00010002],
       ...,
       [0.00010002],
       [0.00010002],
       [0.00010002]])

#### Raise matrix G to the 10th power and then multiply it by the initial PageRank vector. 

In [11]:
%%time
G_10 = np.linalg.matrix_power(G, 10)
#G_10 = G**10
print(G_10.dot(M_int))

[[9.27112230e-05]
 [8.72505665e-05]
 [8.46776745e-05]
 ...
 [1.12693382e-04]
 [9.61683340e-05]
 [8.84430036e-05]]
Wall time: 1min 35s


#### Implement power iteration method for 10 iterations using the sparse matrix HS only. Initialize the PageRank vector to be 1/N for all entries, where N is the total number of webpage in the graph.

In [12]:
%%time
HS = sparse.csr_matrix(H)
HS_pr = HS
pr_2a = M_int

for i in range(10):
    pr_2a = HS_pr.dot(pr_2a)


print(pr_2a)

[[9.22354108e-05]
 [8.50700523e-05]
 [8.25097285e-05]
 ...
 [1.14160942e-04]
 [9.53737830e-05]
 [8.96219574e-05]]
Wall time: 5.85 s


#### Redo this problem but use the matrix H instead of HS.

In [13]:
%%time
H_pr = H
pr_2b = M_int

for i in range(10):
    pr_2b = H_pr.dot(pr_2b)


print(pr_2b)

[[9.22354108e-05]
 [8.50700523e-05]
 [8.25097285e-05]
 ...
 [1.14160942e-04]
 [9.53737830e-05]
 [8.96219574e-05]]
Wall time: 458 ms


In [14]:
MS = sparse.csr_matrix(M2)

#### Initialize the PageRank vector to be 1/N for all entries, where N is the total number of webpage in the graph. 

In [15]:
%%time
e = np.ones((9998,1))
e_t = np.ones((1,9998))
r_final = e/9998

for i in range(10):
    r_temp = beta * MS.dot(r_final)
    S = r_temp.sum()
    r_final = r_temp + ((1-S)/9998)*e

print(r_final)

[[9.27112230e-05]
 [8.72505665e-05]
 [8.46776745e-05]
 ...
 [1.12693382e-04]
 [9.61683340e-05]
 [8.84430036e-05]]
Wall time: 42.7 ms


#### Initialize the PageRank vector using the following formula. 

In [16]:
Q4 = []
for i in range(len(H)):
    temp = [i, np.nonzero(H[:,i])]
    Q4.append(temp)

In [17]:
%%time
r_final = np.ones((9998,1))/9998
r_temp = np.ones((9998,1))/9998
e = np.ones((9998,1))
e_t = np.ones((1,9998))

for num in range(10):
    r_temp = np.ones((9998,1))/9998
    for i in Q4:
        r_temp[i[1]] += beta*r_final[i[0]]/(len(i[1][0]))
    S = r_temp.sum()
    r_final = r_temp + ((1-S)/9998)*e

r_final

Wall time: 1.54 s


array([[9.27112230e-05],
       [8.72505665e-05],
       [8.46776745e-05],
       ...,
       [1.12693382e-04],
       [9.61683340e-05],
       [8.84430036e-05]])

#### Initialize the PageRank using MapReduce

In [18]:
# you may want to import the following libraries
import itertools
import collections

In [19]:

Q5 = []
for i in range(len(H)):
    temp = [i, np.nonzero(H[:,i]),1/9998]
    Q5.append(temp)
Q5_df = np.array(Q5)

<ipython-input-19-093aef08e34e>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Q5_df = np.array(Q5)


#### Define your mapper and reducer function (10 points)

In [20]:
# define your mapper function
def mapper(table):
    node = table[0]
    PR = table[2]
    Dest = table[1][0]
    Degree = len(Dest)
    return [[i,PR/Degree] for i in Dest]
            

# define your reducer function
def reducer(table):
    return (table[0],sum(table[1]))


#### <u> Print the PageRank vector after 10 iterations and runtime. </u> (5 points)

In [22]:
%%time
for num in range(10):
    mapped_values = map(mapper,Q5_df)
    partition_value = list(itertools.chain(*mapped_values))
    partition_data = collections.defaultdict(list)
    for k,v in partition_value:
        partition_data[k].append(v)
    AT = map(reducer,partition_data.items())
    reduced = list(AT)
    reduced.sort()
    reduced_pr = np.array(reduced)[:,1]
    S = reduced_pr.sum()
    r_final = reduced_pr + ((1-S)/9998)*e_t
    Q5_df[:,2] = r_final[0]
    
print(Q5_df[:,2])
    

[9.223541078484978e-05 8.507005225677045e-05 8.25097285025396e-05 ...
 0.00011416094174541629 9.537378299807321e-05 8.962195740879375e-05]
Wall time: 40.9 s


#### Initialize the PageRank vector using MapReduce and multiprocessing

In [23]:
import multiprocessing 
from multiprocessing import Pool
from function_4 import mapper_2
from function_4 import reducer_2
# import your functions. 
# Note that in order to use the multiprocessing library, you need to save all of your user-defined 
# functions into a .py file and import here. If you directly use the functions you have defined in the previous question,
# it will run forever


In [24]:
Q6_df = np.array(Q5)

<ipython-input-24-977efd5ce3b8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Q6_df = np.array(Q5)


In [25]:
Num_core = multiprocessing.cpu_count()

In [26]:
p = multiprocessing.Pool(Num_core)

In [27]:
%%time
for num in range(10):
    mapped_values = p.map(mapper_2,Q6_df)
    partition_value = list(itertools.chain(*mapped_values))
    partition_data = collections.defaultdict(list)
    for k,v in partition_value:
        partition_data[k].append(v)
    AT = p.map(reducer_2,partition_data.items())
    reduced = list(AT)
    reduced.sort()
    reduced_pr = np.array(reduced)[:,1]
    S = reduced_pr.sum()
    r_final = reduced_pr + ((1-S)/9998)*e_t
    Q6_df[:,2] = r_final[0]

print(Q6_df[:,2])

[9.223541078484978e-05 8.507005225677045e-05 8.25097285025396e-05 ...
 0.00011416094174541629 9.537378299807321e-05 8.962195740879375e-05]
Wall time: 1min 39s


#### Compare the runtime of different Q

In [29]:
d = {'Q1':['1min 35s'],'Q2-1':['5.85 s'],'Q2-2':['458 ms'],'Q3':['42.7 ms'],'Q4':['1.534s'],'Q5':['40.9 s'],'Q6':['1min 39s']}
final = pd.DataFrame(d)
final

,Q1,Q2-1,Q2-2,Q3,Q4,Q5,Q6
0,1min 35s,5.85 s,458 ms,42.7 ms,1.534s,40.9 s,1min 39s
